In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")
from PREDICT import PREDICT
from PREDICT.Models import *
from PREDICT.Metrics import *
from PREDICT.Triggers import *
from PREDICT.Plots import *
from Comparison.Detect_Functions import *
import numpy as np
import pandas as pd
from datetime import timedelta
import datetime
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import bambi as bmb

import warnings
warnings.filterwarnings('ignore')

%env PYTENSOR_FLAGS=exception_verbosity=high,floatX=float32

env: PYTENSOR_FLAGS=exception_verbosity=high,floatX=float32


In [3]:

recalthreshold = 0.77 # Paper has AUROC of 0.81, with lower CI at 0.77 

prev_increases = np.arange(1.0001, 1.003, 0.0002).tolist()
undetected = dict({"Static Threshold": 0, "Regular Testing": 0, "SPC3": 0, "SPC5": 0, "SPC7": 0, "Bayesian": 0})
bayes_dict = {"BayesianCoefficients":{}}

# mean and standard deviation for each predictor
# variable at the last visit is used
mean_age, std_age = 62.9, 7.5
mean_bmi, std_bmi = 26.6, 4.4
mean_hip_circ, std_hip_circ = 101.6, 8.8
perc_male, mean_height, std_height = 0.478, 169, 9.2
mean_waist_circ, std_waist_circ = 88.7, 12.7
mean_weight, std_weight = 76.2, 15.2
mean_time_between_visits, std_time_between_visits = 7.3, 2.3

mean_waist_hips_ratio = mean_waist_circ / mean_hip_circ
std_waist_hips_ratio = mean_waist_hips_ratio * np.sqrt(
    (std_waist_circ / mean_waist_circ) ** 2 + (std_hip_circ / mean_hip_circ) ** 2)

# coefficients from non-laboratory logistic regression model
age_at_lv_coef = 0.16 # lv = last visit
bmi_coef = 0.68
hip_circ_coef = -0.05
sex_coef = -0.14
height_coef = -0.15
waist_circ_coef = 0.31
waist_hips_ratio_coef = 0.54
weight_coef = 0.03
time_between_visits_coef = 0.38
bias_coef = -0.74

dm_prev = 0.07  # Initial diabetes prevalence = 7%
startDate = pd.to_datetime('01-06-2019', dayfirst=True) # 01-06-2019
endDate = pd.to_datetime('31-12-2021', dayfirst=True) # 31-12-2021

In [4]:
# Pretrain on year of fake data with no change in prevalence
num_patients = 60
numdays = 365

mydict = {
    'date': list(),
    'outcome': list(),
    'prediction': list(),
    'age': list(),
    'bmi':list(),
    'hip_circ': list(),
    'sex': list(),
    'height': list(),
    'waist_circ': list(),
    'waist_hips_ratio': list(),
    'weight': list(),
    'time_between_visits': list()
}

for i in range(numdays):
    curday = startDate + dt.timedelta(days=i)

    age = np.random.normal(mean_age, std_age, num_patients)
    # min max normalisation
    age = (age - np.min(age)) / (np.max(age) - np.min(age))  # Normalize age to [0, 1]

    bmi = np.random.normal(mean_bmi, std_bmi, num_patients) 
    bmi = (bmi - np.min(bmi)) / (np.max(bmi) - np.min(bmi))  # Normalize BMI to [0, 1]

    hip_circ = np.random.normal(mean_hip_circ, std_hip_circ, num_patients)
    hip_circ = (hip_circ - np.min(hip_circ)) / (np.max(hip_circ) - np.min(hip_circ))

    height = np.random.normal(mean_height, std_height, num_patients)
    height = (height - np.min(height)) / (np.max(height) - np.min(height))  # Normalize height to [0, 1]

    waist_circ = np.random.normal(mean_waist_circ, std_waist_circ, num_patients)
    waist_circ = (waist_circ - np.min(waist_circ)) / (np.max(waist_circ) - np.min(waist_circ))  # Normalize waist circumference to [0, 1]

    waist_hips_ratio = np.random.normal(mean_waist_hips_ratio, std_waist_hips_ratio, num_patients)
    waist_hips_ratio = (waist_hips_ratio - np.min(waist_hips_ratio)) / (np.max(waist_hips_ratio) - np.min(waist_hips_ratio))  # Normalize waist-hips ratio to [0, 1]

    weight = np.random.normal(mean_weight, std_weight, num_patients)
    weight = (weight - np.min(weight)) / (np.max(weight) - np.min(weight))  # Normalize weight to [0, 1]

    time_between_visits = np.random.normal(mean_time_between_visits, std_time_between_visits, num_patients)
    time_between_visits = (time_between_visits - np.min(time_between_visits)) / (np.max(time_between_visits) - np.min(time_between_visits))  # Normalize time between visits to [0, 1]

    sex = np.random.binomial(1, perc_male, num_patients)

    epsilon = np.random.normal(0, 0.2, num_patients) # Simulate error term (mean=0, std=0.2)


    # Calculate baseline log-odds
    lp = bias_coef + age_at_lv_coef * age + bmi_coef * bmi + hip_circ_coef * hip_circ + sex_coef * (sex - perc_male) + height_coef * height + waist_circ_coef * waist_circ  + waist_hips_ratio_coef * waist_hips_ratio + weight_coef * weight  + time_between_visits_coef * time_between_visits
    curpredictions = 1 / (1 + np.exp(-lp))  # Convert to probability

    mod_prob = 1/(1+np.exp(-(lp + epsilon)))
    # intercept changed, but model weights constant
    # diabetes increased as outcome, but not explained by data
    curoutcomes = np.random.binomial(1, mod_prob)           
    # Append to dictionary from the distribution for each of the variables (Table 1)
    mydict['date'].extend([curday] * num_patients)
    mydict['outcome'].extend(curoutcomes)
    mydict['prediction'].extend(curpredictions)
    mydict['age'].extend(age)
    mydict['bmi'].extend(bmi)
    mydict['hip_circ'].extend(hip_circ)
    mydict['sex'].extend(sex)
    mydict['height'].extend(height)
    mydict['waist_circ'].extend(waist_circ)
    mydict['waist_hips_ratio'].extend(waist_hips_ratio)
    mydict['weight'].extend(weight)
    mydict['time_between_visits'].extend(time_between_visits)

pretrain_data = pd.DataFrame(mydict)  

In [7]:
prefit_model = bmb.Model("outcome ~ age + bmi + hip_circ + sex + height + waist_circ + waist_hips_ratio + weight + time_between_visits", pretrain_data, family="bernoulli")
prefit_fitted = prefit_model.fit(
    tune=2000, draws=15000, cores=2, chains=4, target_accept=0.9)

az.summary(prefit_fitted)

Modeling the probability that outcome==1
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [Intercept, age, bmi, hip_circ, sex, height, waist_circ, waist_hips_ratio, weight, time_between_visits]


Output()

Sampling 4 chains for 2_000 tune and 15_000 draw iterations (8_000 + 60_000 draws total) took 755 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,-0.605,0.089,-0.771,-0.436,0.0,0.0,128176.0,45118.0,1.0
age,0.167,0.061,0.050,0.280,0.0,0.0,128434.0,44428.0,1.0
bmi,0.715,0.061,0.600,0.831,0.0,0.0,126019.0,42061.0,1.0
height,-0.139,0.061,-0.254,-0.025,0.0,0.0,123775.0,44479.0,1.0
hip_circ,-0.060,0.061,-0.173,0.057,0.0,0.0,122714.0,45941.0,1.0
sex,-0.188,0.027,-0.239,-0.136,0.0,0.0,126204.0,44669.0,1.0
time_between_visits,0.369,0.061,0.254,0.484,0.0,0.0,125252.0,43394.0,1.0
waist_circ,0.256,0.061,0.144,0.370,0.0,0.0,128463.0,43294.0,1.0
waist_hips_ratio,0.541,0.061,0.430,0.660,0.0,0.0,131707.0,44373.0,1.0
weight,-0.043,0.061,-0.155,0.074,0.0,0.0,131356.0,44883.0,1.0


In [5]:
bayesian_priors = {
    "Intercept": (-0.605, 0.089), 
    "age": (0.167, 0.061), 
    "bmi": (0.715, 0.061), 
    "hip_circ": (-0.06, 0.061),
    "sex": (-0.188, 0.027), 
    "height":(-0.139, 0.061), 
    "waist_circ":(0.256, 0.061),
    "waist_hips_ratio":(0.541, 0.061), 
    "weight":(-0.043, 0.061), 
    "time_between_visits":(0.369, 0.061)}

In [6]:
# Get bootstrap OE with CI
preds = -0.605 + pretrain_data.age*0.167 + pretrain_data.bmi*0.715 + pretrain_data.hip_circ*(-0.06) +\
        pretrain_data.sex*(-0.188) + pretrain_data.height*(-0.139) + pretrain_data.waist_circ*0.256 +\
        pretrain_data.waist_hips_ratio*0.541 + pretrain_data.weight*(-0.043) + pretrain_data.time_between_visits*0.369
preds = 1 / (1 + np.exp(-preds))
outcome = pretrain_data['outcome'].values
for i in range(1000):
    boot_indices = np.random.choice(range(len(outcome)), size=len(outcome), replace=True)
    boot_outcome = outcome[boot_indices]
    boot_preds = preds[boot_indices]
    boot_oe = boot_outcome.mean() / boot_preds.mean()
    if i == 0:
        oe_values = [boot_oe]
    else:
        oe_values.append(boot_oe)
        
print(f"Pretrain OE: {np.mean(oe_values)} with std: {np.std(oe_values)} and 95% CI: {np.percentile(oe_values, 2.5)} - {np.percentile(oe_values, 97.5)}")

Pretrain OE: 0.994173383632472 with std: 0.006155547317111696 and 95% CI: 0.9827990690664817 - 1.005972261145391


In [7]:
resultsloc = "./Results/simulation/outcome_prevalence"
os.makedirs(resultsloc, exist_ok=True)
if not os.path.exists(os.path.join(resultsloc, 'performance_metrics.csv')):
    header = pd.DataFrame(columns=['Time', 'Accuracy', 'AUROC', 'Precision', 'CalibrationSlope', 'CITL',
    'OE', 'AUPRC', 'F1Score', 'impact_or_prev', 'Method', 'Data_Type'])
    header.to_csv(os.path.join(resultsloc, 'performance_metrics.csv'), index=False)

In [ ]:
for prev_increase in prev_increases:
    regular_ttd = []
    static_ttd = []
    spc_ttd3 = []
    spc_ttd5 = []
    spc_ttd7 = []
    bayesian_ttd = []
    mydict = {
            'date': list(),
            'outcome': list(),
            'prediction': list(),
            'age': list(),
            'bmi':list(),
            'hip_circ': list(),
            'sex': list(),
            'height': list(),
            'waist_circ': list(),
            'waist_hips_ratio': list(),
            'weight': list(),
            'time_between_visits': list()
        }

    num_patients = 60
    numdays = (endDate - startDate).days
    
    for i in range(numdays):
        curday = startDate + dt.timedelta(days=i)

        age = np.random.normal(mean_age, std_age, num_patients)
        # min max normalisation
        age = (age - np.min(age)) / (np.max(age) - np.min(age))  # Normalize age to [0, 1]

        bmi = np.random.normal(mean_bmi, std_bmi, num_patients) 
        bmi = (bmi - np.min(bmi)) / (np.max(bmi) - np.min(bmi))  # Normalize BMI to [0, 1]

        hip_circ = np.random.normal(mean_hip_circ, std_hip_circ, num_patients)
        hip_circ = (hip_circ - np.min(hip_circ)) / (np.max(hip_circ) - np.min(hip_circ))

        height = np.random.normal(mean_height, std_height, num_patients)
        height = (height - np.min(height)) / (np.max(height) - np.min(height))  # Normalize height to [0, 1]

        waist_circ = np.random.normal(mean_waist_circ, std_waist_circ, num_patients)
        waist_circ = (waist_circ - np.min(waist_circ)) / (np.max(waist_circ) - np.min(waist_circ))  # Normalize waist circumference to [0, 1]

        waist_hips_ratio = np.random.normal(mean_waist_hips_ratio, std_waist_hips_ratio, num_patients)
        waist_hips_ratio = (waist_hips_ratio - np.min(waist_hips_ratio)) / (np.max(waist_hips_ratio) - np.min(waist_hips_ratio))  # Normalize waist-hips ratio to [0, 1]

        weight = np.random.normal(mean_weight, std_weight, num_patients)
        weight = (weight - np.min(weight)) / (np.max(weight) - np.min(weight))  # Normalize weight to [0, 1]

        time_between_visits = np.random.normal(mean_time_between_visits, std_time_between_visits, num_patients)
        time_between_visits = (time_between_visits - np.min(time_between_visits)) / (np.max(time_between_visits) - np.min(time_between_visits))  # Normalize time between visits to [0, 1]

        sex = np.random.binomial(1, perc_male, num_patients)

        epsilon = np.random.normal(0, 0.2, num_patients) # Simulate error term (mean=0, std=0.2)
        

        # Calculate baseline log-odds
        lp = bias_coef + age_at_lv_coef * age + bmi_coef * bmi + hip_circ_coef * hip_circ + sex_coef * (sex - perc_male) + height_coef * height + waist_circ_coef * waist_circ  + waist_hips_ratio_coef * waist_hips_ratio + weight_coef * weight  + time_between_visits_coef * time_between_visits
        curpredictions = 1 / (1 + np.exp(-lp))  # Convert to probability

        # Generate outcomes to simulate diabetes rates increasing over time
        if i % 30 == 0:
            dm_prev *= prev_increase # this increases the probability by x% each month

        mod_prob = 1/(1+np.exp(-(lp + dm_prev  + epsilon)))
        # intercept changed, but model weights constant
        # diabetes increased as outcome, but not explained by data
        curoutcomes = np.random.binomial(1, mod_prob)           
        

        # Append to dictionary from the distribution for each of the variables (Table 1)
        mydict['date'].extend([curday] * num_patients)
        mydict['outcome'].extend(curoutcomes)
        mydict['prediction'].extend(curpredictions)
        mydict['age'].extend(age)
        mydict['bmi'].extend(bmi)
        mydict['hip_circ'].extend(hip_circ)
        mydict['sex'].extend(sex)
        mydict['height'].extend(height)
        mydict['waist_circ'].extend(waist_circ)
        mydict['waist_hips_ratio'].extend(waist_hips_ratio)
        mydict['weight'].extend(weight)
        mydict['time_between_visits'].extend(time_between_visits)
        

    df = pd.DataFrame(mydict)  
    ########################################### Baseline Testing #######################################
    model = RecalibratePredictions()
    model.trigger = TimeframeTrigger(model=model, updateTimestep=900, dataStart=df['date'].min(), dataEnd=df['date'].max())
    mytest = PREDICT(data=df, model=model, startDate='min', endDate='max', timestep='month')
    mytest.addLogHook(Accuracy(model))
    mytest.addLogHook(AUROC(model))
    mytest.addLogHook(Precision(model))
    mytest.addLogHook(CalibrationSlope(model))
    mytest.addLogHook(CITL(model))
    mytest.addLogHook(OE(model))
    mytest.addLogHook(AUPRC(model))
    mytest.run()
    log = mytest.getLog()

    baseline_metrics = pd.DataFrame({'Time': list(log["Accuracy"].keys()), 'Accuracy': list(log["Accuracy"].values()), 'AUROC': list(log["AUROC"].values()), 'Precision': list(log["Precision"].values()), 'CalibrationSlope': list(log["CalibrationSlope"].values()), 'CITL': list(log["CITL"].values()), 'OE': list(log["O/E"].values()), 'AUPRC': list(log["AUPRC"].values()), 'impact_or_prev': list([str(prev_increase)] * len(log["Accuracy"])), 'Method':list(['Baseline'] * len(log["Accuracy"]))})
    ########################################### Save Metrics #######################################
    baseline_metrics["Data_Type"] = "Outcome Prevalence Simulation"

    baseline_metrics.to_csv(os.path.join(resultsloc, 'performance_metrics.csv'), mode='a', header=False, index=False)
    # Get OE thresholds for static recal from original model
    recalthreshold_lower = 0.994173383632472 - 3*0.006155547317111696
    recalthreshold_upper = 0.994173383632472 + 3*0.006155547317111696
    print(f"Using OE Threshold of {recalthreshold_lower} - {recalthreshold_upper} for impact {dm_prev}")
    
    ########################################### Recalibration Methods Testing #######################################      
    out_prev_metrics_df = get_metrics_recal_methods(df, dm_prev, recalthreshold_lower, recalthreshold_upper, model_name='Outcome_prev_datasim')
    undetected, regular_ttd, static_ttd, spc_ttd3, spc_ttd5, spc_ttd7 = run_recalibration_tests(df, startDate, undetected, regular_ttd, static_ttd, spc_ttd3, spc_ttd5, spc_ttd7, recalthreshold_lower, recalthreshold_upper)
    
    
    ########################################### Bayesian Testing #######################################
    bayes_coef_ci = {
        key: (bayesian_priors[key][0] - 3 * bayesian_priors[key][1], bayesian_priors[key][0] + 3 * bayesian_priors[key][1])
        for key in bayesian_priors
    }
    
    bay_model = BayesianModel(input_data=df, 
                            model_formula = "outcome ~ age + bmi + hip_circ + sex + height + waist_circ + waist_hips_ratio + weight + time_between_visits",
                            priors=bayesian_priors, verbose=False, draws=10000, tune=2000, chains=4, cores=2)
    bay_model.trigger = AlwaysTrigger(model=bay_model)
    mytest = PREDICT(data=df, model=bay_model, startDate='min', endDate='max', timestep='month')
    mytest.addLogHook(Accuracy(bay_model))
    mytest.addLogHook(AUROC(bay_model))
    mytest.addLogHook(Precision(bay_model))
    mytest.addLogHook(CalibrationSlope(bay_model))
    mytest.addLogHook(CITL(bay_model))
    mytest.addLogHook(OE(bay_model))
    mytest.addLogHook(AUPRC(bay_model))
    mytest.addLogHook(TrackBayesianCoefs(bay_model))
    mytest.run()
    log = mytest.getLog()

    if "BayesianCoefficients" in log:
        bayes_dict["BayesianCoefficients"].update(log["BayesianCoefficients"])
    
    ttd = find_bayes_coef_change(bayes_dict["BayesianCoefficients"], detectDate=startDate, undetected=undetected, threshold=0.1)
    bayesian_ttd.append(ttd)

    bayes_metrics = pd.DataFrame({'Time': list(log["Accuracy"].keys()), 'Accuracy': list(log["Accuracy"].values()), 'AUROC': list(log["AUROC"].values()), 'Precision': list(log["Precision"].values()), 'CalibrationSlope': list(log["CalibrationSlope"].values()), 'CITL': list(log["CITL"].values()), 'OE': list(log["O/E"].values()), 'AUPRC': list(log["AUPRC"].values()), 'impact_or_prev': list([str(dm_prev)] * len(log["Accuracy"])), 'Method':list(['Bayesian'] * len(log["Accuracy"]))})
    
    ########################################### Save Metrics #######################################

    # concatenate all the dataframes into one
    out_prev_metrics_df = pd.concat([out_prev_metrics_df, bayes_metrics], ignore_index=True)
    out_prev_metrics_df["Data_Type"] = "Outcome Prevalence Simulation"

    out_prev_metrics_df.to_csv(os.path.join(resultsloc, 'performance_metrics.csv'), mode='a', header=False, index=False)

    update_ttd_table(regular_ttd, static_ttd, spc_ttd3, spc_ttd5, spc_ttd7, bayesian_ttd, prev_increase, os.path.join(resultsloc, 'output_prev_ttd_tbl.csv'))
    
    # Generate plots
    plot_incidence_over_time(df, None, regular_ttd, static_ttd, spc_ttd3, spc_ttd5, spc_ttd7, bayesian_ttd, 'outcome_prev'+str(prev_increase), fileloc=resultsloc)
    BayesianCoefsPlot(bayes_dict, 'outcome_prev'+str(prev_increase))

plot_time_to_detect('output_prev_ttd_tbl.csv', 'outcome_prev')
    

        



Using OE Threshold of 0.9757067416811369 - 1.012640025583807 for impact 0.07044941412093295


Modeling the probability that outcome==1


Model formula is set to:  outcome ~ age + bmi + hip_circ + sex + height + waist_circ + waist_hips_ratio + weight + time_between_visits


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [Intercept, age, bmi, hip_circ, sex, height, waist_circ, waist_hips_ratio, weight, time_between_visits]


Output()

...

Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 261 seconds.
Modeling the probability that outcome==1
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [Intercept, age, bmi, hip_circ, sex, height, waist_circ, waist_hips_ratio, weight, time_between_visits]


Output()

Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 241 seconds.
Modeling the probability that outcome==1
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [Intercept, age, bmi, hip_circ, sex, height, waist_circ, waist_hips_ratio, weight, time_between_visits]


Output()

Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 265 seconds.
